In [83]:
# Kaggle NFL Big Data Bowl 2022 Submission

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Read data into memory
games_df = pd.read_csv('input/games.csv')
players_df = pd.read_csv('input/players.csv')
plays_df = pd.read_csv('input/plays.csv')
pff_scouting_df = pd.read_csv('input/PFFScoutingData.csv')

plays_df.info()
plays_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19979 entries, 0 to 19978
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gameId                  19979 non-null  int64  
 1   playId                  19979 non-null  int64  
 2   playDescription         19979 non-null  object 
 3   quarter                 19979 non-null  int64  
 4   down                    19979 non-null  int64  
 5   yardsToGo               19979 non-null  int64  
 6   possessionTeam          19979 non-null  object 
 7   specialTeamsPlayType    19979 non-null  object 
 8   specialTeamsResult      19979 non-null  object 
 9   kickerId                19878 non-null  float64
 10  returnerId              6938 non-null   object 
 11  kickBlockerId           100 non-null    float64
 12  yardlineSide            19813 non-null  object 
 13  yardlineNumber          19979 non-null  int64  
 14  gameClock               19979 non-null

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
0,2018090600,37,"J.Elliott kicks 65 yards from PHI 35 to end zone, Touchback.",1,0,0,PHI,Kickoff,Touchback,44966.0,...,NaN,NaN,NaN,0,0,NaN,66.0,NaN,40,45
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Center-R.Lovato. J.Hardy to ATL 41 for 5 yards (K.Grugier-Hill). PENALTY on PHI-S.Gibson, Unsportsmanlike Conduct, 15 yards, enforced at ATL 41.",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,NaN,56.0,5.0,36,18
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Center-J.Overbaugh, Holder-M.Bosher.",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13
3,2018090600,677,"M.Bosher kicks 64 yards from ATL 35 to PHI 1. S.Gibson to PHI 31 for 30 yards (D.Kazee, D.Riley).",1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, Center-R.Lovato, Touchback.",1,4,18,PHI,Punt,Touchback,45603.0,...,NaN,NaN,NaN,0,3,NaN,65.0,NaN,45,45


In [108]:
# Distinct result types
plays_df[plays_df['specialTeamsPlayType'] == 'Punt']['specialTeamsResult'].unique()
plays_df[plays_df['specialTeamsPlayType'] == 'Punt']['playResult'].unique()

array([ 36,  45,  49,  58,  61,  38,  37,  44,  46,  25,  47,  59,  48,
        -2, -25,  40,  41,  54,  43,  31,  28,  62,  35,  33,  22,  57,
        34,  39,  53,  29,  50,  42,  56,  19,  23,  30,  52,  67,  65,
        15,  55,  21,  26,  32,  51,  16,  12, -26,  24, -15,  63,  27,
        64,  -9,  10,  68,  -3,  60, -28,  -7,   0,  70,  -4,  69, -11,
         4,  -1,  20,  14,   2, -21,   6,  -6,  72,   8, -45,  13,  17,
        66,  80,  11, -16,  18,   9,   7,   3, -18,  -8,  -5, -22, -34,
        82, -12, -33,  71, -17, -19, -27,   5,   1, -10, -14,  73, -40,
       -32, -20, -30, -36, -29])

In [77]:
# Non special teams includes trick plays, fumbled snaps, etc...
plays_df[plays_df['specialTeamsResult'] == 'Non-Special Teams Result'].head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
43,2018090900,2616,"(13:57) (Punt formation) C.Bojorquez FUMBLES (Aborted) at BUF 27, and recovers at BUF 14. C.Bojorquez to BUF 14 for no gain (T.Bowser).",3,4,8,BUF,Punt,Non-Special Teams Result,NaN,...,NaN,NaN,NaN,26,0,NaN,NaN,NaN,-25,49
274,2018090910,2976,(Kick formation) TWO-POINT CONVERSION ATTEMPT. M.Palardy rushes left end. ATTEMPT FAILS.,4,0,0,CAR,Extra Point,Non-Special Teams Result,NaN,...,NaN,NaN,NaN,16,0,NaN,NaN,NaN,0,25
944,2018092304,391,(9:17) (Punt formation) Direct snap to C.Davis. C.Grant left end to JAX 46 for no gain (L.Sims).,1,4,4,JAX,Punt,Non-Special Teams Result,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,64
1087,2018092309,2100,"(:03) (Field Goal formation) J.Scott Aborted. H.Bradley FUMBLES at WAS 43, recovered by GB-J.Scott at GB 49. J.Scott pass incomplete short right to M.Crosby.",2,1,10,GB,Field Goal,Non-Special Teams Result,NaN,...,NaN,NaN,NaN,28,10,I,NaN,NaN,0,67
1251,2018092700,2608,(11:03) (Punt formation) J.Hekker pass incomplete deep right to J.Reynolds.,3,4,10,LA,Punt,Non-Special Teams Result,NaN,...,NaN,NaN,NaN,28,20,I,NaN,NaN,0,74


In [100]:
# Muffed punts
plays_df[(plays_df['specialTeamsResult'] == 'Muffed') & (plays_df['specialTeamsPlayType'] == 'Punt')][
    ['playDescription', 'kickReturnYardage']].head(20)

,playDescription,kickReturnYardage
18,"(:34) M.Bosher punts 38 yards to PHI 32, Center-J.Overbaugh. T.Sullivan MUFFS catch, touched at PHI 32, RECOVERED by ATL-K.Ishmael at PHI 32. K.Ishmael to PHI 32 for no gain (L.Reynolds).",NaN
40,"(3:01) (Punt formation) C.Bojorquez punts 52 yards to BAL 21, Center-R.Ferguson. J.Grant MUFFS catch, ball out of bounds at BAL 15.",NaN
173,"(4:41) T.Daniel punts 47 yards to NE 17, Center-J.Weeks. R.McCarron MUFFS catch, RECOVERED by HOU-J.Bademosi at NE 16. J.Bademosi to NE 16 for no gain (J.Jones).",NaN
237,"(:54) (Punt formation) L.Cooke punts 41 yards to NYG 19, Center-C.Tinker. K.Clay MUFFS catch, RECOVERED by JAX-D.Payne at NYG 21. D.Payne to NYG 21 for no gain (K.Clay).",NaN
367,"(13:37) S.Koch punts 55 yards to CIN 20, Center-M.Cox. D.Phillips MUFFS catch, and recovers at CIN 15. D.Phillips to CIN 31 for 16 yards (J.Allen).",16.0
437,"(9:08) D.Kaser punts 58 yards to BUF 8, Center-M.Windt. M.Murphy MUFFS catch, recovered by BUF-T.Jones at BUF 0. Touchback (U.Nwosu). BUF-T.Jones was injured during the play. His return is Questionable. Touchback due to penalty in end zone. T.Jones assisted to tunnel. PENALTY on LAC-U.Nwosu, Unnecessary Roughness, 15 yards, enforced at BUF 20.",NaN
438,"(4:04) D.Kaser punts 45 yards to BUF 49, Center-M.Windt. M.Murphy MUFFS catch, and recovers at LAC 48. M.Murphy to LAC 42 for 6 yards (A.Phillips; N.Dzubnar).",6.0
803,"(2:21) B.Colquitt punts 3 yards to CLE 34, Center-C.Hughlett. T.Brooks MUFFS catch, ball out of bounds at CLE 28. 56 Pierre-Louis deflected punt",NaN
919,"(2:14) T.Daniel punts 57 yards to NYG 15, Center-J.Weeks. S.Coley MUFFS catch, and recovers at NYG 16.",0.0
1012,"(12:33) (Punt formation) M.Haack punts 42 yards to OAK 19, Center-J.Denney. J.Nelson MUFFS catch, touched at OAK 19, and recovers at OAK 17. J.Nelson to OAK 17 for no gain (W.Aikens).",0.0


In [102]:
# Players that have caught punts
punt_plays_df = plays_df[plays_df['specialTeamsPlayType'] == 'Punt'].dropna(subset=['returnerId'])
punt_plays_df['primaryReturnerId'] = punt_plays_df['returnerId'].str.split(';').str[0].astype('int64')
punt_returns_df = pd.merge(punt_plays_df, players_df, left_on='primaryReturnerId', right_on='nflId')
punt_returns_df.head()[['gameId', 'playDescription', 'specialTeamsResult', 'returnerId', 'nflId', 'displayName']]

,gameId,playDescription,specialTeamsResult,returnerId,nflId,displayName
0,2018090600,"(9:20) C.Johnston punts 56 yards to ATL 36, Center-R.Lovato. J.Hardy to ATL 41 for 5 yards (K.Grugier-Hill). PENALTY on PHI-S.Gibson, Unsportsmanlike Conduct, 15 yards, enforced at ATL 41.",Return,42450,42450,Justin Hardy
1,2018090600,"(12:33) C.Johnston punts 38 yards to ATL 20, Center-R.Lovato, fair catch by J.Hardy.",Fair Catch,42450,42450,Justin Hardy
2,2018091600,"(12:57) M.Palardy punts 48 yards to ATL 42, Center-J.Jansen, fair catch by J.Hardy.",Fair Catch,42450,42450,Justin Hardy
3,2018091600,"(5:43) M.Palardy punts 37 yards to ATL 10, Center-J.Jansen, fair catch by J.Hardy.",Fair Catch,42450,42450,Justin Hardy
4,2018092300,"(9:01) T.Morstead punts 57 yards to ATL 13, Center-Z.Wood. J.Hardy pushed ob at ATL 19 for 6 yards (T.Hill; J.Hardee).",Return,42450,42450,Justin Hardy


In [81]:
# Distribution of punts caught by player
punt_returners_grouped_df = punt_returns_df.groupby(['primaryReturnerId', 'displayName']).size().reset_index(
    name='puntsCaught')
punt_returners_grouped_df.sort_values('puntsCaught', ascending=False).head(20)

,primaryReturnerId,displayName,puntsCaught
12,35527,Andre Roberts,133
71,43663,Alex Erickson,119
85,44932,Tarik Cohen,100
50,42794,DeAndre Carter,93
131,46279,Braxton Berrios,83
79,44837,Jabrill Peppers,83
41,42051,Diontae Spencer,80
113,46116,Christian Kirk,79
59,43406,Pharoh Cooper,77
68,43556,Chester Rogers,74


In [107]:
# Group by return result (positive, neutral, negative)
conditions = [
    (punt_returns_df['kickReturnYardage'] > 0),
    (punt_returns_df['kickReturnYardage'] == 0) | (punt_returns_df['kickReturnYardage'].isna()),
    (punt_returns_df['kickReturnYardage'] < 0),
]
result_types = ['positive', 'neutral', 'negative']
punt_returns_df['returnResult'] = np.select(conditions, result_types)
punt_returns_df.groupby(['primaryReturnerId', 'displayName', 'returnResult']).size()

primaryReturnerId  displayName                  returnResult
29830              Adam Jones                   negative         1
                                                neutral          2
                                                positive         5
29957              Darren Sproles               neutral         16
                                                positive        20
31517              Tramon Williams              neutral         10
                                                positive        10
32202              Ted Ginn                     neutral          5
                                                positive         3
33097              Dominique Rodgers-Cromartie  neutral          1
33117              Jordy Nelson                 neutral          3
                                                positive         1
33130              DeSean Jackson               neutral          2
                                                positive         5
3

In [110]:
# Add isTouchdown column
punt_returns_df['isTouchdown'] = punt_returns_df['playDescription'].str.contains('TOUCHDOWN')

,gameId,playDescription,specialTeamsResult,kickReturnYardage,returnResult,isTouchdown,displayName
0,2018090600,"(9:20) C.Johnston punts 56 yards to ATL 36, Center-R.Lovato. J.Hardy to ATL 41 for 5 yards (K.Grugier-Hill). PENALTY on PHI-S.Gibson, Unsportsmanlike Conduct, 15 yards, enforced at ATL 41.",Return,5.0,positive,False,Justin Hardy
1,2018090600,"(12:33) C.Johnston punts 38 yards to ATL 20, Center-R.Lovato, fair catch by J.Hardy.",Fair Catch,NaN,neutral,False,Justin Hardy
2,2018091600,"(12:57) M.Palardy punts 48 yards to ATL 42, Center-J.Jansen, fair catch by J.Hardy.",Fair Catch,NaN,neutral,False,Justin Hardy
3,2018091600,"(5:43) M.Palardy punts 37 yards to ATL 10, Center-J.Jansen, fair catch by J.Hardy.",Fair Catch,NaN,neutral,False,Justin Hardy
4,2018092300,"(9:01) T.Morstead punts 57 yards to ATL 13, Center-Z.Wood. J.Hardy pushed ob at ATL 19 for 6 yards (T.Hill; J.Hardee).",Return,6.0,positive,False,Justin Hardy
5,2018092300,"(10:11) T.Morstead punts 49 yards to ATL 26, Center-Z.Wood. J.Hardy to ATL 32 for 6 yards (C.Robertson). PENALTY on NO-J.Hardee, Face Mask (15 Yards), 15 yards, enforced at ATL 32. Penalty on NO-Z.Wood, Offensive Holding, declined.",Return,6.0,positive,False,Justin Hardy
6,2018093000,"(7:47) K.Huber punts 31 yards to ATL 8, Center-C.Harris, fair catch by J.Hardy.",Fair Catch,NaN,neutral,False,Justin Hardy
7,2018100707,"(10:31) (Punt formation) J.Berry punts 51 yards to ATL 18, Center-K.Canaday. J.Hardy to ATL 21 for 3 yards (B.Allen).",Return,3.0,positive,False,Justin Hardy
8,2018100707,"(2:21) (Punt formation) J.Berry punts 55 yards to ATL 11, Center-K.Canaday. J.Hardy to ATL 11 for no gain (C.Sutton).",Return,0.0,neutral,False,Justin Hardy
9,2018101400,"(4:53) B.Anger punts 37 yards to ATL 14, Center-G.Sanborn. J.Hardy MUFFS catch, and recovers at ATL 14. J.Hardy to ATL 14 for no gain (J.Watson). Penalty on ATL-J.Bethel, Running Into the Kicker, declined.",Muffed,0.0,neutral,False,Justin Hardy
